In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [6]:
# Initialize the driver
driver = webdriver.Chrome()

In [7]:
driver.get('https://www.sheglam.com/product/Color-Bloom-Liquid-Blush-Love-Cake-7957494104229')

In [8]:
html_data = BeautifulSoup(driver.page_source,'html.parser')

In [9]:
reviews = html_data.find_all('div',{'class':'fun-pc-sg-review-item-wrapper'})

In [23]:
for review in reviews:
    name = review.find('div', {'class': 'fun-pc-sg-review-item-name'}).text
    print(name.strip())

    rating_date = review.find('div', {'class': 'fun-pc-sg-review-item-date'}).text
    print(rating_date.strip())

    comment = review.find('div', {'class': 'fun-pc-sg-review-item-content'}).text
    print(comment.strip())

    rating_div = review.find('div', {'class': 'fun-pc-std-rate-wrapper'})  # Find rating wrapper

    # Extract the rating percentage
    rating_span = rating_div.find('span', {'class': 'fun-pc-std-rate-active'}) if rating_div else None
    rating_percentage = rating_span['style'].split(':')[-1].replace('%', '').replace(';', '').strip() if rating_span else None

    # Convert percentage to a 5-star rating
    rating = round(float(rating_percentage) / 20, 1) if rating_percentage else 'N/A'

    print(f"Rating: {rating}")  # Show as stars
    print('-------------')

b***m
2024.12.29
Leaves a very nice tone on the face
Rating: 5.0
-------------
s***5
2024.02.06
Nunca había probado un producto como este! Me encantó demasiado. Y este color “hush hush” más todavía.
Rating: 5.0
-------------
k***h
2023.12.13
Me gusta mucho el colorete porque me gusta más el rosa además se ve más tierno con ese color
Rating: 5.0
-------------
G***o
2023.07.02
Im in love qith this product for real :)
Rating: 5.0
-------------
E***g
2023.03.15
I have never used a cream blush before, and i was a little apprehensive. However, it goes on so well and give the perfect amount of color. !
Rating: 5.0
-------------


In [34]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

names = []
ratings = []
rating_dates = []
comments = []

# Initialize the driver
driver = webdriver.Chrome()

try:
    driver.get('https://www.sheglam.com/product/Jelly-Licious-Hydrating-Lip-Blush-Tint-Ahoy-8769544388773')

    # Wait for the reviews section to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "fun-pc-sg-review-item-wrapper"))
    )

    page = 1  # Start from page 1
    last_page_scraped = False  # Flag to track if last page is scraped

    while True:  # Loop until there are no more pages
        print(f"Scraping Page {page}...")

        # Parse the current page
        html_data = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = html_data.find_all('div', {'class': 'fun-pc-sg-review-item-wrapper'})

        for review in reviews:
            # Extract the reviewer's name
            name = review.find('div', {'class': 'fun-pc-sg-review-item-name'}).text.strip()
            print(name)
            names.append(name)

            # Extract the rating date
            rating_date = review.find('div', {'class': 'fun-pc-sg-review-item-date'}).text.strip()
            print(rating_date)
            rating_dates.append(rating_date)

            # Extract the review comment
            comment = review.find('div', {'class': 'fun-pc-sg-review-item-content'}).text.strip()
            print(comment)
            comments.append(comment)

            # **Extract rating properly** from all stars
            rating_div = review.find('div', {'class': 'fun-pc-std-rate-wrapper'})  # Find rating wrapper

            if rating_div:
                # Find all stars
                active_stars = rating_div.find_all('span', {'class': 'fun-pc-std-rate-active'})

                total_stars = 5  # Ratings are always out of 5 stars
                filled_stars = 0  # Counter for total rating

                for star in active_stars:
                    style_attr = star.get("style", "")
                    if "width" in style_attr:
                        width_value = int(style_attr.split("width: ")[1].split("%")[0])  # Extract width percentage
                        filled_stars += width_value / 100  # Convert percentage to decimal (100% -> 1.0, 50% -> 0.5)

                # Compute final rating
                rating = round(filled_stars, 1)  # Ensures correct decimal rounding
            else:
                rating = 'N/A'  # Handle cases where no rating is found

            print(f"Rating: {rating}")
            ratings.append(rating)
            print('-------------')

        # **Check if the "Next" button is disabled (Stopping Condition)**
        disabled_next_button = driver.find_elements(By.XPATH, "//div[contains(@class, 'fun-pc-std-pagination-disabled') and @aria-label='next page']")

        if disabled_next_button:
            if not last_page_scraped:  # If last page hasn't been processed, do one final scrape
                print(f"Final check: Scraping last page {page + 1}...")
                last_page_scraped = True  # Set flag to avoid infinite loop
                continue  # Force one more iteration to scrape the last page
            else:
                print("Last page reached. Scraping completed.")
                break  # Stop pagination

        # **Click the "Next" button**
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//div[@class='fun-pc-std-pagination-arrow' and @aria-label='next page']"))
            )

            print(f"Clicking Next Page {page + 1}...")

            # Scroll into view to ensure visibility
            driver.execute_script("arguments[0].scrollIntoView();", next_button)

            # Use JavaScript click to handle potential click issues
            driver.execute_script("arguments[0].click();", next_button)

            # Wait for new content to load before continuing
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "fun-pc-sg-review-item-wrapper"))
            )

            page += 1  # Increase page counter
        except Exception as e:
            print(f"Could not click next page: {e}")
            break

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    driver.quit()

Scraping Page 1...
b***m
2024.12.29
It leaves a beautiful color on the lips. I love it.
Rating: 5.0
-------------
s***1
2024.12.10
Love it! Perfect color
Rating: 5.0
-------------
I***.
2024.08.22
Jelly-Licious Hydrating Lip & Blush Tint-Cheerio
Rating: 5.0
-------------
N***A
2024.11.22
El color me encantó todo, pero al darle rosca, la gelatina se ha salido del envase...Y lo use dos veces.
Rating: 2.0
-------------
Final check: Scraping last page 2...
Scraping Page 1...
b***m
2024.12.29
It leaves a beautiful color on the lips. I love it.
Rating: 5.0
-------------
s***1
2024.12.10
Love it! Perfect color
Rating: 5.0
-------------
I***.
2024.08.22
Jelly-Licious Hydrating Lip & Blush Tint-Cheerio
Rating: 5.0
-------------
N***A
2024.11.22
El color me encantó todo, pero al darle rosca, la gelatina se ha salido del envase...Y lo use dos veces.
Rating: 2.0
-------------
Last page reached. Scraping completed.


In [35]:
import pandas as pd 
data = pd.DataFrame({'name':names,
                    'rating_date':rating_dates,
                    'rating':ratings,
                    'comments':comments,})

In [23]:
len(names)

420

In [24]:
data

,name,rating_date,rating,comments
0,b***m,2024.12.29,5.0,Leaves a very nice tone on the face
1,s***5,2024.02.06,5.0,Nunca había probado un producto como este! Me ...
2,k***h,2023.12.13,5.0,Me gusta mucho el colorete porque me gusta más...
3,G***o,2023.07.02,5.0,Im in love qith this product for real :)
4,E***g,2023.03.15,5.0,"I have never used a cream blush before, and i ..."
...,...,...,...,...
415,L***.,2022.10.26,3.0,"I have an olive complexion, this color was a s..."
416,D***n,2022.10.17,3.0,I tried this as soon as I got it and I loved t...
417,S***A,2022.10.11,3.0,I like the concept of this blush. Hush hush wa...
418,J***s,2022.08.17,3.0,I really love the formula of this blush. I too...


In [36]:
import pandas as pd

# Save the dataframe to a CSV file with a semicolon separator
data.to_csv(r'C:\Users\PC HP 1\Desktop\data_externe\reviews4.csv', sep=';', index=False)

In [19]:
print(data.iloc[400])

name                                                       M***3
rating_date                                           2024.02.25
rating                                                       3.0
comments       its a solid pink type of blush yet i wished it...
Name: 400, dtype: object
